## Python Statistics Essential Training - 02_05_challenge

In [1]:
#meta 3/29/2020
#Ch2. Challenge: Tuberculosis Analytics
#my code and solution code bits

In [2]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as pp

%matplotlib inline

#  Tuberculosis Analytics

The data set comes from the World Health Organization and it records the number of confirmed tuberculosis cases by country, year, and demographic group. The demographic groups are broken down by sex and age. 

This data comes untidy. The sex and the age groups are recorded somewhat confusingly in the column names. For instance, MO4 stands for male from zero to four years of age. Also, there are many missing values, which appeared as NAN, not a number. 

The challenge: transform this data frame into this clean version. Here, only actual observations are recorded and the age range and sex are recorded as separate variables.

Data structure:
- country
- year
- age 
- sex 
- cases


## 0. Load Data

In [3]:
df = pd.read_csv("lynda_data/tb.csv")
print(df.shape)
df.head()

(5769, 22)


,country,year,m04,m514,m014,m1524,m2534,m3544,m4554,m5564,...,f04,f514,f014,f1524,f2534,f3544,f4554,f5564,f65,fu
0,AD,1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AD,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AD,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AD,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AD,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns

Index(['country', 'year', 'm04', 'm514', 'm014', 'm1524', 'm2534', 'm3544',
       'm4554', 'm5564', 'm65', 'mu', 'f04', 'f514', 'f014', 'f1524', 'f2534',
       'f3544', 'f4554', 'f5564', 'f65', 'fu'],
      dtype='object')

In [5]:
df.isnull().sum()

country      23
year          0
m04        5377
m514       5368
m014       3388
m1524      3362
m2534      3361
m3544      3354
m4554      3348
m5564      3355
m65        3361
mu         5496
f04        5376
f514       5365
f014       3391
f1524      3375
f2534      3369
f3544      3371
f4554      3368
f5564      3375
f65        3373
fu         5497
dtype: int64

## 1. EDA
Missing data - some records don't have a country designation.  

In [6]:
df[df['country'].isnull()].tail()

,country,year,m04,m514,m014,m1524,m2534,m3544,m4554,m5564,...,f04,f514,f014,f1524,f2534,f3544,f4554,f5564,f65,fu
3709,NaN,2004,NaN,NaN,31.0,319.0,1092.0,866.0,371.0,159.0,...,NaN,NaN,30.0,400.0,819.0,554.0,203.0,106.0,74.0,NaN
3710,NaN,2005,NaN,NaN,98.0,355.0,1027.0,874.0,365.0,146.0,...,NaN,NaN,105.0,399.0,809.0,525.0,213.0,95.0,91.0,NaN
3711,NaN,2006,NaN,NaN,86.0,347.0,1052.0,799.0,386.0,174.0,...,NaN,NaN,74.0,485.0,875.0,521.0,239.0,92.0,80.0,NaN
3712,NaN,2007,NaN,NaN,57.0,370.0,1018.0,786.0,346.0,149.0,...,NaN,NaN,69.0,417.0,826.0,513.0,242.0,102.0,76.0,NaN
3713,NaN,2008,NaN,NaN,30.0,387.0,1033.0,757.0,346.0,149.0,...,NaN,NaN,73.0,466.0,702.0,437.0,226.0,110.0,80.0,0.0


Sum of all cases

In [7]:
df[['m04', 'm514', 'm014', 'm1524', 'm2534', 'm3544',
       'm4554', 'm5564', 'm65', 'mu', 'f04', 'f514', 'f014', 'f1524', 'f2534',
       'f3544', 'f4554', 'f5564', 'f65', 'fu']].sum()

m04         3452.0
m514       11619.0
m014      189142.0
m1524    2220132.0
m2534    3134962.0
m3544    2911442.0
m4554    2381091.0
m5564    1680559.0
m65      1467533.0
mu          1559.0
f04         2522.0
f514       15831.0
f014      252879.0
f1524    1816417.0
f2534    2054610.0
f3544    1406662.0
f4554     961304.0
f5564     661171.0
f65       611717.0
fu           691.0
dtype: float64

In [8]:
df[['m04', 'm514', 'm014', 'm1524', 'm2534', 'm3544',
       'm4554', 'm5564', 'm65', 'mu', 'f04', 'f514', 'f014', 'f1524', 'f2534',
       'f3544', 'f4554', 'f5564', 'f65', 'fu']].sum().sum()

21785295.0

Preview data - records for males from 0 to 4 years old

In [9]:
#confirm nulls
print("Nulls: ", len(df[df['m04'].isnull()]))
print("Not nulls: ", len(df[df['m04'].notnull()]))

print("Zero count: ", len(df[df['m04']==0]))
print("Non-zero count: ", len(df[df['m04']>0]))


df[df['m04']> 0].head()

Nulls:  5377
Not nulls:  392
Zero count:  244
Non-zero count:  148


,country,year,m04,m514,m014,m1524,m2534,m3544,m4554,m5564,...,f04,f514,f014,f1524,f2534,f3544,f4554,f5564,f65,fu
142,AL,2006,1.0,4.0,5.0,24.0,19.0,22.0,21.0,19.0,...,0.0,2.0,2.0,12.0,8.0,7.0,7.0,7.0,13.0,0.0
144,AL,2008,1.0,0.0,1.0,23.0,26.0,13.0,19.0,13.0,...,0.0,1.0,1.0,20.0,10.0,8.0,5.0,5.0,10.0,NaN
170,AM,2005,1.0,2.0,3.0,170.0,104.0,83.0,84.0,30.0,...,0.0,3.0,3.0,27.0,21.0,10.0,11.0,4.0,7.0,0.0
242,AR,2006,19.0,48.0,67.0,519.0,484.0,360.0,351.0,346.0,...,18.0,56.0,74.0,438.0,437.0,235.0,197.0,173.0,213.0,NaN
243,AR,2007,14.0,63.0,77.0,656.0,623.0,401.0,415.0,389.0,...,14.0,56.0,70.0,558.0,500.0,246.0,217.0,172.0,246.0,NaN


Total # of cases for males between 0 and 4 years old

In [10]:
df[df['m04']> 0]['m04'].sum()

3452.0

## 2. Clean Data

In [11]:
df_tidy = df.melt(['country', 'year'], ['m04', 'm514', 'm014', 'm1524', 'm2534', 'm3544',
       'm4554', 'm5564', 'm65', 'mu', 'f04', 'f514', 'f014', 'f1524', 'f2534',
       'f3544', 'f4554', 'f5564', 'f65', 'fu'], "sexage", "cases")
print(df_tidy.shape)
df_tidy.head()

(115380, 4)


,country,year,sexage,cases
0,AD,1989,m04,NaN
1,AD,1990,m04,NaN
2,AD,1991,m04,NaN
3,AD,1992,m04,NaN
4,AD,1993,m04,NaN


In [12]:
df_tidy.isnull().sum()

country      460
year           0
sexage         0
cases      79630
dtype: int64

In [13]:
#validate sum of all cases (must equal to the number above) - 21785295
df_tidy['cases'].sum()

21785295.0

In [14]:
#separate sex and age
df_tidy['sex'] = df_tidy['sexage'].str.slice(0,1)
df_tidy['age_temp'] = df_tidy['sexage'].str.slice(1)
df_tidy.head()

,country,year,sexage,cases,sex,age_temp
0,AD,1989,m04,NaN,m,04
1,AD,1990,m04,NaN,m,04
2,AD,1991,m04,NaN,m,04
3,AD,1992,m04,NaN,m,04
4,AD,1993,m04,NaN,m,04


In [15]:
#note: see solution alternative
df_tidy['age'] = df_tidy['age_temp'].apply(lambda s: '0-4' if s=='04' else('65+' if s=='65' else ('unk' if s=='u' else s[:-2] + '-' + s[-2:])))
df_tidy.head()

,country,year,sexage,cases,sex,age_temp,age
0,AD,1989,m04,NaN,m,04,0-4
1,AD,1990,m04,NaN,m,04,0-4
2,AD,1991,m04,NaN,m,04,0-4
3,AD,1992,m04,NaN,m,04,0-4
4,AD,1993,m04,NaN,m,04,0-4


In [16]:
df_tidy.drop(['age_temp', 'sexage'], axis=1, inplace=True)

In [17]:
#clean up
df_tidy_final = df_tidy.dropna(subset=['cases'])
print(df_tidy_final.shape)
df_tidy_final.head()

(35750, 5)


,country,year,cases,sex,age
15,AD,2005,0.0,m,0-4
16,AD,2006,0.0,m,0-4
18,AD,2008,0.0,m,0-4
42,AE,2006,0.0,m,0-4
43,AE,2007,0.0,m,0-4


In [31]:
df_tidy_final.sort_values(['country','year','age','sex'])
df_tidy_final.tail(14)

,country,year,cases,sex,age
114880,TV,2008,0.0,f,unk
114936,UA,2006,0.0,f,unk
114937,UA,2007,0.0,f,unk
114965,UG,2008,0.0,f,unk
114994,US,2008,0.0,f,unk
115023,UY,2008,0.0,f,unk
115050,UZ,2007,0.0,f,unk
115109,VE,2008,0.0,f,unk
115166,VN,2008,0.0,f,unk
115195,VU,2008,0.0,f,unk


Are all unknown age cases = 0?  We only want to keep the column if it has some cases.

In [28]:
df_tidy[(df_tidy['age']=='unk') & (df_tidy['cases']>0)]['cases'].sum()

2250.0

Validate counts: all and subgroups

In [22]:
#validate sum of all cases (must equal to the number above) - 21785295
print("Sum all: ", df_tidy_final.groupby(['age','sex'])['cases'].sum().sum())
df_tidy_final.groupby(['age','sex'])['cases'].sum()

Sum all:  21785295.0


age    sex
0-14   f       252879.0
       m       189142.0
0-4    f         2522.0
       m         3452.0
15-24  f      1816417.0
       m      2220132.0
25-34  f      2054610.0
       m      3134962.0
35-44  f      1406662.0
       m      2911442.0
45-54  f       961304.0
       m      2381091.0
5-14   f        15831.0
       m        11619.0
55-64  f       661171.0
       m      1680559.0
65+    f       611717.0
       m      1467533.0
unk    f          691.0
       m         1559.0
Name: cases, dtype: float64

In [20]:
df_tidy_final[(df_tidy_final['sex']=="m") & (df_tidy_final['age']=="0-4")]['cases'].sum()

3452.0

## Xtra


In [32]:
#xtra note: atlernatives

#my alt - separate sex and age
#df_tidy["sex"] = df_tidy['sexage].apply(lambda s: s[0])
#df_tidy['age_temp']=df_tidy['sexage'].apply(lambda s: s[1:])

#Solution alternative - manually map age
#df_tidy['age'] = df_tidy['age_temp'].map({'04': '0-4', '514': '5-14', '1524': '15-24', '2534': '25-34', '3544': '35-44','4554': '45-54', '5564': '55-64', '65': '65+', 'u': np.nan})



Compare with final solution:  
If group by age and sex, lost some cases due to 1) not mapping 0-14 age and 2) disqualifying unknown age cases.

In [33]:
df_solution = pd.read_csv('lynda_data/tb_solution.csv')
df_solution.head()

,country,year,age,sex,cases
0,AD,1996,15-24,f,1.0
1,AD,1996,15-24,m,0.0
2,AD,1996,25-34,f,1.0
3,AD,1996,25-34,m,0.0
4,AD,1996,35-44,f,0.0


In [34]:
#validate sum of all cases (must equal to the number above) - 21785295
df_solution['cases'].sum()

21785295.0

In [35]:
df_solution.groupby(['age','sex'])['cases'].sum().sum()

21341024.0

In [36]:
df_solution[df_solution['age'].isnull()]['cases'].sum()

444271.0